In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.layers import Input, Embedding, Flatten, Dot, Dense
from keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import keras
from sklearn.decomposition import PCA
import seaborn as sns
from keras.optimizers import Adam
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

from keras.models import load_model

from sklearn.metrics import mean_absolute_error

In [ ]:
def change_number(unique_lst):
    unique = pd.DataFrame(unique_lst).reset_index()
    indexes = unique['index']
    ids = unique[0]
    tuple_zip = list(zip(ids, indexes))
    dic = dict((x, y) for x, y in tuple_zip)
    reverse_dic = dict((y, x) for x, y in tuple_zip)
    return dic, reverse_dic

In [ ]:
ratings = pd.read_csv('result_breakfastMF.csv')

In [ ]:
unique_users = list(ratings['user_id'].unique())
unique_recipe = list(ratings['recipe_id'].unique())

In [ ]:
dic_user, dummy = change_number(unique_users)

ratings['user_id'] = ratings['user_id'].replace(dic_user)
                                                

In [ ]:
dic_recipe, dic_recipe_reverse = change_number(unique_recipe)

ratings['recipe_id'] = ratings['recipe_id'].replace(dic_recipe)

### Scale 

In [ ]:
scaler = MinMaxScaler()
ratings['rating'] = ratings['rating'].values.astype(float)
rating_scaled = pd.DataFrame(scaler.fit_transform(ratings['rating'].values.reshape(-1,1)))
ratings['rating'] = rating_scaled

### Train test split 

In [ ]:
train, test = train_test_split(ratings, test_size=0.2)

In [ ]:
n_users = ratings.user_id.nunique()
print('number of users is', n_users)
n_recipe = ratings.recipe_id.nunique()
print('number of recipes is:', n_recipe)

### Make model

In [ ]:
n_latent_factors_user = 20
n_latent_factors_recipe = 20

recipe_input = keras.layers.Input(shape=[1],name='Recipe')
recipe_embedding = keras.layers.Embedding(n_recipe + 1, n_latent_factors_recipe, name='Recipe-Embedding')(recipe_input)
recipe_vec = keras.layers.Flatten(name='Flatten-Recipes')(recipe_embedding)
recipe_vec = keras.layers.Dropout(0.2)(recipe_vec)


user_input = keras.layers.Input(shape=[1],name='User')
user_embedding = Embedding(n_users+1, n_latent_factors_user, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)
user_vec = keras.layers.Flatten(name='Flatten-Users')(user_embedding)
user_vec = keras.layers.Dropout(0.2)(user_vec)


concat = keras.layers.concatenate([recipe_vec, user_vec], axis=1, name='Concat')
concat_dropout = keras.layers.Dropout(0.2)(concat)
dense = keras.layers.Dense(200,name='FullyConnected')(concat)
dropout_1 = keras.layers.Dropout(0.2,name='Dropout')(dense)
dense_2 = keras.layers.Dense(100,name='FullyConnected-1')(concat)
dropout_2 = keras.layers.Dropout(0.2,name='Dropout')(dense_2)
dense_3 = keras.layers.Dense(50,name='FullyConnected-2')(dense_2)
dropout_3 = keras.layers.Dropout(0.2,name='Dropout')(dense_3)
dense_4 = keras.layers.Dense(20,name='FullyConnected-3', activation='relu')(dense_3)


result = keras.layers.Dense(1, activation='relu',name='Activation')(dense_4)
adam = Adam(lr=0.005)
model = keras.Model([user_input, recipe_input], result)
model.compile(optimizer=adam,loss= 'mean_absolute_error')

In [ ]:
SVG(model_to_dot(model,  show_shapes=True, show_layer_names=True, rankdir='HB').create(prog='dot', format='svg'))

In [ ]:
history = model.fit([train.user_id, train.recipe_id], train.rating, epochs=100, verbose=1)
model.save('NN_Breakfast.h5')

In [ ]:
pd.Series(history.history['loss']).plot(logy=True)
plt.xlabel("Epoch")
plt.ylabel("Train Error")

In [ ]:
y_hat = np.round(model.predict([test.user_id, test.recipe_id]),0)
y_true = test.rating

MAE = mean_absolute_error(y_true, y_hat)

percentage_wrong = MAE/1 * 100
percentage_wrong
MAE